In [17]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
data_set = pd.read_csv("C:/Users/AbdulRahman/Desktop/grad_folders/MAIN_GRAD/Travia/Implementation/AI/DataSet/lateset Dataset Main/pre_processed_df_with.csv")    

In [4]:
data_set.head()

,latitude,longitude,stars,review_count,attributes_RestaurantsPriceRange2,attributes_GoodForKids,attributes_Ambience_touristy,attributes_Ambience_romantic,attributes_Ambience_intimate,attributes_Ambience_trendy,...,hours_Sunday,attributes_RestaurantsGoodForGroups,attributes_RestaurantsAttire,attributes_NoiseLevel,attributes_GoodForMeal_dessert,attributes_GoodForMeal_latenight,attributes_GoodForMeal_lunch,attributes_GoodForMeal_dinner,attributes_GoodForMeal_brunch,attributes_GoodForMeal_breakfast
0,39.955505,-75.155564,4.0,80,1,1,0,0,0,0,...,7:0-21:0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,39.953949,-75.143226,4.0,245,2,0,0,0,0,0,...,13:30-22:0,True,'casual',u'average',True,NaN,NaN,True,NaN,NaN
2,39.943223,-75.162568,4.5,205,1,1,0,0,0,0,...,NaN,False,u'casual',u'quiet',False,False,True,True,False,False
3,40.006707,-75.289671,3.5,109,1,1,0,0,0,0,...,11:0-0:0,True,u'casual',u'average',False,True,False,True,False,False
4,39.962582,-75.135657,3.5,65,2,1,0,0,0,0,...,11:0-20:0,NaN,NaN,NaN,False,False,False,False,False,False


# feature selection

In [8]:
ds = data_set[
    [
        "stars",

        "attributes_RestaurantsPriceRange2",

        # good for kids or romantic
        "attributes_GoodForKids", # option1

        "attributes_Ambience_romantic", # option2
        "attributes_Ambience_intimate", # option2

        "attributes_Ambience_touristy",

        # trendy or classy or casual 
        "attributes_Ambience_trendy",
        "attributes_Ambience_classy",
        "attributes_Ambience_casual",

        # choose one or many from those 
        "Bars_Night",
        "Beauty_Health_Care",
        "Cafes",
        "GYM",
        "Restaurants_Cuisines",
        "Shops"
    ]
]

In [9]:
ds.head()

,stars,attributes_RestaurantsPriceRange2,attributes_GoodForKids,attributes_Ambience_romantic,attributes_Ambience_intimate,attributes_Ambience_touristy,attributes_Ambience_trendy,attributes_Ambience_classy,attributes_Ambience_casual,Bars_Night,Beauty_Health_Care,Cafes,GYM,Restaurants_Cuisines,Shops
0,4.0,1,1,0,0,0,0,0,1,0,0,1,0,1,0
1,4.0,2,0,0,0,0,0,1,0,0,0,0,0,1,0
2,4.5,1,1,0,0,0,0,0,1,0,0,0,0,1,0
3,3.5,1,1,0,0,0,0,0,1,0,0,0,0,1,0
4,3.5,2,1,0,0,0,0,1,0,0,0,0,0,1,0


In [13]:
def create_user_pref_vector(user_pref):
    user_vector = np.zeros(15, dtype=np.float32)
    user_vector[0] = user_pref["min_stars"]
    user_vector[1] = user_pref["price_range"]
    if user_pref["intimate_or_family"] == "family":
        user_vector[2] = 1
    else:
        user_vector[4] = 1
        user_vector[5] = 1
    user_vector[3] = user_pref["attributes_Ambience_touristy"]
    user_vector[6] = user_pref["attributes_Ambience_trendy"]
    user_vector[7] = user_pref["attributes_Ambience_classy"]
    user_vector[8] = user_pref["attributes_Ambience_casual"]

    for cat in user_pref["category"]:
        user_vector[9 + cat] = 1
    return user_vector

In [14]:
user_pref = {
    #"Bars_Night","Beauty_Health_Care","Cafes","GYM","Restaurants_Cuisines","Shops"
    "category": [0,2,4],
    "min_stars": 3.5,
    "price_range": 3,
    "attributes_Ambience_touristy": 0,
    "attributes_Ambience_trendy": 1,
    "attributes_Ambience_classy": 0,
    "attributes_Ambience_casual": 1,
    "intimate_or_family": 'family'
}

In [18]:
v = create_user_pref_vector(user_pref)

In [19]:
v.reshape(1, -1)

array([[3.5, 3. , 1. , 0. , 0. , 0. , 1. , 0. , 1. , 1. , 0. , 1. , 0. ,
        1. , 0. ]], dtype=float32)

In [32]:
def recommend_businesses(user_pref, data, original_df, k=3):
    user_vector = create_user_pref_vector(user_pref)    
    similarities = cosine_similarity(data, user_vector.reshape(1, -1)).flatten()
    top_k_unsorted = np.argpartition(-similarities, k)[:k]
    sorted_indices = top_k_unsorted[np.argsort(similarities[top_k_unsorted])[::-1]]
    recommendations = original_df.iloc[sorted_indices][[
        "name", "stars", "attributes_RestaurantsPriceRange2", "address", "city","attributes_GoodForKids","attributes_Ambience_romantic","attributes_Ambience_intimate","attributes_Ambience_touristy","attributes_Ambience_trendy","attributes_Ambience_classy","attributes_Ambience_casual",
        "Bars_Night", "Beauty_Health_Care", "Cafes", "GYM", "Restaurants_Cuisines", "Shops"
    ]].copy()
    recommendations["similarity"] = similarities[sorted_indices]
    return recommendations

In [33]:
recommend_businesses(user_pref,ds,data_set)

,name,stars,attributes_RestaurantsPriceRange2,address,city,attributes_GoodForKids,attributes_Ambience_romantic,attributes_Ambience_intimate,attributes_Ambience_touristy,attributes_Ambience_trendy,attributes_Ambience_classy,attributes_Ambience_casual,Bars_Night,Beauty_Health_Care,Cafes,GYM,Restaurants_Cuisines,Shops,similarity
15430,The Melting Pot,3.5,3,294 Lamp And Lantern Vlg,Chesterfield,0,1,1,0,1,0,1,1,0,0,0,1,0,0.963303
14446,Malbec Argentine Steakhouse,4.0,3,400-402 S 2nd St,Philadelphia,0,1,1,0,1,1,1,1,0,0,0,1,0,0.948200
7826,N 7,4.0,3,1117 Montegut St,New Orleans,0,1,1,0,1,1,1,1,0,0,0,1,0,0.948200


In [30]:
user_pref = {
    #"Bars_Night","Beauty_Health_Care","Cafes","GYM","Restaurants_Cuisines","Shops"
    "category": [0],
    "min_stars": 3.5,
    "price_range": 3,
    "attributes_Ambience_touristy": 1,
    "attributes_Ambience_trendy": 1,
    "attributes_Ambience_classy": 0,
    "attributes_Ambience_casual": 1,
    "intimate_or_family": 'intimate'
}